In [1]:
import torch

from cnp.output import MeanFieldGaussianLayer

In [2]:
mfg_layer = MeanFieldGaussianLayer()

In [3]:
B = 1
T = 7
C = 2

tensor = torch.rand(B, T, C)

mfg_layer.mean_and_cov(tensor)

(tensor([[0.0819, 0.0120, 0.3108, 0.7026, 0.9424, 0.0059, 0.1319]]),
 tensor([[[1.2708, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.7033, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.2926, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.7612, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.8764, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7337, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0648]]]),
 tensor([[[1.9640, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 1.3964, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.9857, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 1.4544, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 1.5695, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4268, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0

In [4]:
B = 1
T = 7
C = 2

tensor = torch.rand(B, T, C)
y_target = torch.rand(B, T, 1)

mfg_layer.loglik(tensor, y_target)

tensor(-8.5482, grad_fn=<MeanBackward0>)

In [6]:
B = 1
T = 7
C = 2
num_samples = 3

tensor = torch.rand(B, T, C)
y_target = torch.rand(B, T, 1)

mfg_layer.sample(tensor, num_samples=num_samples).shape

torch.Size([3, 1, 7])